In [1]:
import pandas
import numpy
import scipy.stats
from statsmodels.formula.api import ols
import requests_cache
import xlrd
from matplotlib import pyplot as plt
plt.style.use('seaborn-poster')
import datetime

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def get_shiller(url="http://www.econ.yale.edu/~shiller/data/ie_data.xls"):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    excel = session.get(url, stream=True)
    # must be wrapped in a BytesIO for read_excel to accept it...
    df = pandas.read_excel(pandas.io.common.BytesIO(excel.content),
                           sheet_name='Data',
                           engine='xlrd',
                           skiprows=7,
                           skipfooter=11, # WARN: is this always 11?
                           index_col=0,
                           parse_dates=True)
    df.rename(columns={'P' : 'Price',
                      'D' : 'Dividend',
                      'E' : 'Earnings',
                      'Price' : 'Real Price',
                      'Dividend' : 'Real Dividend',
                      'Price.1' : 'Real Total Return Price',
                      'Earnings' : 'Real Earnings',
                      'Earnings.1' : 'Real Total Return Scaled Earnings'}, inplace=True)
    del(df['Unnamed: 13'], df['Unnamed: 15']) # garbage columns from Shiller's spreadsheet
    return df

shiller = get_shiller()
shiller.head()

In [ ]:
shiller['Div Yield'] = shiller['Dividend'] / shiller['Price'] / 12
shiller['MoM Price'] = shiller['Price'] / shiller['Price'].shift(1)
shiller['MoM TR'] = shiller['MoM Price'] + shiller['Div Yield']
shiller['CPI Change'] = shiller['CPI'] / shiller['CPI'].shift(1)
shiller['Real MoM TR'] = shiller['MoM TR'] / shiller['CPI Change'] - 1
shiller['Mean CAPE'] = shiller['CAPE'].expanding().mean()
shiller.head()